# [ LG에너지솔루션 DX Expert 양성과정 - Transformer #1]

Transformer의 encoder 구조인 BERT를 활용한 text classification

## 강의 복습
강의자료: Transformer
- `Transformer`: Transformer는 Attention의 병렬적 사용을 통해 효율적인 학습이 가능한 구조의 언어 모델. Sequence를 입력으로 받아 sequenc를 출력하는 구조
- `BERT`: Bidirectional Encoder Representations from Transformers의 약자로, 양방향 인코더"

<img src="https://github.com/hwk0702/2024_LG_ES_External_Lecture/blob/main/240703_Transformer/image/Transformer01.png?raw=true" width="400">

## 실습 요약
1. 본 실습에서는 Transformer의 encoder만을 활용한 BERT를 통해 자연어 분류를 수행합니다.

---

## Import modules

In [ ]:
!pip install transformers==4.29.2

In [ ]:
# 기본 라이브러리 임포트
import json  # JSON 파일을 읽고 쓰기 위한 라이브러리
import os    # 운영체제와 상호작용하기 위한 라이브러리
import re    # 정규 표현식을 사용하기 위한 라이브러리
import random  # 난수 생성을 위한 라이브러리
import numpy as np  # 수치 연산을 위한 라이브러리
import pandas as pd  # 데이터 처리를 위한 라이브러리
from collections import defaultdict  # 딕셔너리의 기본 값을 설정하기 위한 라이브러리

from time import time  # 시간 측정을 위한 라이브러리
from tqdm.auto import tqdm  # 반복문 진행 상황을 시각적으로 보여주기 위한 라이브러리

# 사이킷런 라이브러리 임포트
from sklearn.model_selection import train_test_split  # 데이터셋을 학습 및 테스트 세트로 분리
from sklearn.metrics import roc_auc_score, f1_score, recall_score, precision_score  # 다양한 평가 지표

# Gensim 라이브러리 임포트
import gensim  # 자연어 처리에 사용되는 라이브러리

# PyTorch 라이브러리 임포트
import torch  # PyTorch 메인 라이브러리
import torch.nn as nn  # 신경망 모듈
import torch.nn.functional as F  # 신경망 함수
from torch.utils.data import Dataset, DataLoader  # 데이터셋 및 데이터로더

# Hugging Face Transformers 라이브러리 임포트
from transformers import AutoTokenizer, BertForSequenceClassification, AdamW  # BERT 모델 및 토크나이저

# 시각화 라이브러리 임포트
import matplotlib.pyplot as plt  # 데이터 시각화를 위한 라이브러리
import seaborn as sns  # 데이터 시각화를 위한 고급 라이브러리

# Seaborn의 플롯 스타일 설정
custom_params = {"axes.spines.right": False, "axes.spines.top": False}
sns.set_theme(style="ticks", rc=custom_params)

import warnings  # 경고 메시지를 관리하기 위한 모듈
warnings.filterwarnings("ignore")  # 경고 메시지를 무시하도록 설정

## Functions

In [ ]:
def torch_seed(random_seed):
    torch.manual_seed(random_seed)
    torch.cuda.manual_seed(random_seed)
    torch.cuda.manual_seed_all(random_seed) # if use multi-GPU
    # CUDA randomness
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

    np.random.seed(random_seed)
    random.seed(random_seed)
    os.environ['PYTHONHASHSEED'] = str(random_seed)

def calc_metrics(y_true: list, y_score: np.ndarray, y_pred: list) -> dict:
    auroc = roc_auc_score(y_true, y_score)
    f1 = f1_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)

    return {
        'auroc'    : auroc,
        'f1'       : f1,
        'recall'   : recall,
        'precision': precision
    }

def train(model, dataloader, optimizer, log_interval: int, device: str) -> list:

    correct = 0
    total = 0
    total_loss = 0
    total_score = []
    total_preds = []
    total_targets = []

    model.train()

    for i, batch in enumerate(dataloader):
        inputs, attn_mask, targets = batch

        # convert device
        inputs, attn_mask, targets = inputs.to(device), attn_mask.to(device), targets.to(device)

        # model outputs
        outputs = model(inputs, attention_mask=attn_mask, labels=targets)

        # outputs
        loss = outputs[0]
        logit = outputs[1]

        # calculate gradients
        loss.backward()

        # update model weights
        optimizer.step()
        optimizer.zero_grad()

        # total loss and acc
        total_loss += loss.item()
        preds = logit.argmax(dim=1)

        correct += targets.eq(preds).sum().item()
        total += targets.size(0)

        total_score.extend(logit.cpu().tolist())
        total_preds.extend(preds.cpu().tolist())
        total_targets.extend(targets.cpu().tolist())

        # log learning history
        if i % log_interval == 0 or (i+1) == len(dataloader):
            print('TRAIN [%5d/%5d]: Loss: %.3f | Acc: %.3f%% [%5d/%5d]' %
                 (i+1, len(dataloader), total_loss/(i+1), 100.*correct/total, correct, total))

    # calculate metrics
    metrics = calc_metrics(
        y_true  = total_targets,
        y_score = np.array(total_score)[:,1],
        y_pred  = total_preds
    )

    metrics.update([('acc',correct/total), ('loss',total_loss/len(dataloader))])

    # logging metrics
    print('\nTRAIN: Loss: %.3f | Acc: %.3f%% | AUROC: %.3f%% | F1-Score: %.3f%% | Recall: %.3f%% | Precision: %.3f%%\n' %
         (metrics['loss'], 100.*metrics['acc'], 100.*metrics['auroc'], 100.*metrics['f1'], 100.*metrics['recall'], 100.*metrics['precision']))


    return metrics

def test(model, dataloader, log_interval: int, device: str) -> list:

    correct = 0
    total = 0
    total_loss = 0
    total_score = []
    total_preds = []
    total_targets = []

    torch_seed(223)
    model.eval()

    with torch.no_grad():
        for i, batch in enumerate(dataloader):
            inputs, attn_mask, targets = batch

            # convert device
            inputs, attn_mask, targets = inputs.to(device), attn_mask.to(device), targets.to(device)

            # model outputs
            outputs = model(inputs, attention_mask=attn_mask, labels=targets)

            # outputs
            loss = outputs[0]
            logit = outputs[1]

            # total loss and acc
            total_loss += loss.item()
            preds = logit.argmax(dim=1)

            correct += targets.eq(preds).sum().item()
            total += targets.size(0)

            total_score.extend(logit.cpu().tolist())
            total_preds.extend(preds.cpu().tolist())
            total_targets.extend(targets.cpu().tolist())

            # log learning history
            if i % log_interval == 0 or (i+1) == len(dataloader):
                print('TEST [%5d/%5d]: Loss: %.3f | Acc: %.3f%% [%5d/%5d]' %
                      (i+1, len(dataloader), total_loss/(i+1), 100.*correct/total, correct, total))

    # calculate metrics
    metrics = calc_metrics(
        y_true  = total_targets,
        y_score = np.array(total_score)[:,1],
        y_pred  = total_preds
    )

    metrics.update([('acc',correct/total), ('loss',total_loss/len(dataloader))])

    # logging metrics
    print('\nTEST: Loss: %.3f | Acc: %.3f%% | AUROC: %.3f%% | F1-Score: %.3f%% | Recall: %.3f%% | Precision: %.3f%%\n' %
         (metrics['loss'], 100.*metrics['acc'], 100.*metrics['auroc'], 100.*metrics['f1'], 100.*metrics['recall'], 100.*metrics['precision']))

    # return
    return metrics


def fit(
    model, trainloader, testloader, optimizer, epochs: int, log_interval: int, device: str) -> list:

    train_history = defaultdict(list)
    test_history = defaultdict(list)

    # fitting model
    for i in range(epochs):
        print(f'\nEpoch: [{i+1}/{epochs}]')
        train_metrics = train(
            model        = model,
            dataloader   = trainloader,
            optimizer    = optimizer,
            log_interval = log_interval,
            device       = device,
        )

        test_metrics = test(
            model        = model,
            dataloader   = testloader,
            log_interval = log_interval,
            device       = device
        )

        # stack history
        for k, v in train_metrics.items():
            train_history[k].append(v)
        for k, v in test_metrics.items():
            test_history[k].append(v)

    return train_history, test_history

# Configuration for experiments

In [ ]:
class Config:
    """
    실험 설정을 담고 있는 클래스
    """

    # dataset 관련 parameters
    datadir = './data'            # 데이터가 저장된 디렉토리
    dataname = 'naver_shopping.txt' # 데이터 파일 이름
    train_ratio = 0.6             # 학습 데이터셋의 비율
    max_seq_length = 100          # 입력 시퀀스의 최대 길이

    # model
    pretrained_name = 'klue/bert-base' # 사전 학습된 BERT 모델 이름

    # training 관련 parameters
    epochs = 5                    # 총 학습 에폭 수
    batch_size = 32               # 학습 데이터 배치 크기
    test_batch_size = 128         # 테스트 데이터 배치 크기
    learning_rate = 0.00001       # 학습률
    num_workers = 2               # 데이터 로딩을 위한 워커 수
    log_interval = 100            # 로그 출력 간격

    # device
    device = 'cuda'               # 학습을 위한 디바이스 (cuda 또는 cpu)

    # seed
    seed = 223                    # 시드 값 (재현성 확보를 위해 고정)

# Config 클래스의 인스턴스를 생성
cfg = Config()

## Load dataset and dataloader

네이버 쇼핑 리뷰 데이터셋 [[download](https://github.com/bab2min/corpus/tree/master/sentiment)]

In [ ]:
!wget -P ./data https://raw.githubusercontent.com/bab2min/corpus/master/sentiment/naver_shopping.txt

In [ ]:
with open(os.path.join(cfg.datadir, cfg.dataname), 'r', encoding='utf-8') as f:
    data = f.readlines()

In [ ]:
data[:5]

### Preprocessing

In [ ]:
# 데이터 처리
rate = []       # 평점을 저장할 리스트
label = []      # 레이블을 저장할 리스트
sentence = []   # 리뷰 텍스트를 저장할 리스트

# 리뷰 데이터를 순회하며 처리
for cur_review in tqdm(data):
    # 평점과 리뷰 텍스트를 탭(\t)으로 분리
    cur_review = cur_review.split('\t')
    
    # 평점을 리스트에 추가
    rate.append(cur_review[0])
    
    # 리뷰 텍스트를 리스트에 추가 (줄바꿈 문자 제거)
    sentence.append(cur_review[1].strip('\n'))
    
    # 평점에 따라 레이블을 할당
    # 평점 4, 5: 긍정 (Positive)
    # 평점 1, 2, 3: 부정 (Negative)
    if int(cur_review[0]) > 3:
        label.append(0)  # 긍정 레이블
    else:
        label.append(1)  # 부정 레이블

In [ ]:
# 첫 다섯 개의 리뷰 텍스트 출력
print(sentence[:5])

In [ ]:
# 텍스트 전처리 함수
def preprocess(text):
    # 특수 문자 제거
    text = re.sub('[-=+,#/\?:^$~@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]', '', text)
    # 특정 한글 문자 제거 (예: ㅠ, ㅎ, ㅋ)
    text = re.sub('[ㅠㅎㅋ]', '', text)
    return text

# 정규화된 리뷰 텍스트 리스트
normalize_sentence = []

# 리뷰 텍스트를 순회하며 전처리
for s_i in tqdm(sentence):
    s_i = preprocess(s_i)  # 전처리 함수 적용
    normalize_sentence.append(s_i)  # 전처리된 텍스트 추가

In [ ]:
print(normalize_sentence[:5])

**Tokenizer: 입력된 텍스트를 모델에서 처리할 수 있는 데이터로 변환하는 것**

In [ ]:
# 사전 학습된 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained(cfg.pretrained_name)

# 토큰화할 문장 예시 출력
print('Sentence')
print(normalize_sentence[1])

# 문장 토큰화
print('\nTokenized sentence')
print(tokenizer.tokenize(normalize_sentence[1]))

In [ ]:
# 샘플링 전 데이터셋 크기 출력
print('Before sampling')
print('len(label): ', len(label))
print('len(normalize_sentence): ', len(normalize_sentence))

# 데이터셋 크기 설정
dataset_size = len(label)

# 샘플링할 인덱스 선택
torch_seed(cfg.seed)
select_idx = np.random.choice(range(dataset_size), size=dataset_size // 2)

# 샘플링된 데이터셋 생성
label = np.array(label)[select_idx]
normalize_sentence = np.array(normalize_sentence)[select_idx]

# 샘플링 후 데이터셋 크기 출력
print('\nAfter sampling')
print('len(label): ', len(label))
print('len(normalize_sentence): ', len(normalize_sentence))

In [ ]:
# 레이블 분포 시각화
x, y = np.unique(label, return_counts=True)

# 막대 그래프 생성
palette = ['red', 'green']  # 부정적인 레이블과 긍정적인 레이블의 색상 지정
ax = sns.barplot(x=x, y=y, palette=palette)
plt.title('Label Distribution')
plt.ylabel('Frequency')
plt.xticks(ticks=[0, 1], labels=['Negative', 'Positive'])

# 막대 그래프의 각 막대에 라벨 추가
ax.bar_label(ax.containers[0], fmt='%d')
ax.bar_label(ax.containers[1], fmt='%d')

# 그래프 출력
plt.show()

### Split train and test

In [ ]:
# 데이터셋 크기 설정
dataset_size = len(label)

# 시드 설정
torch_seed(cfg.seed)

# 학습 데이터와 테스트 데이터의 인덱스를 분할
train_idx, test_idx = train_test_split(np.arange(dataset_size), train_size=cfg.train_ratio)

# 학습 데이터 설정
x_train = normalize_sentence[train_idx]
y_train = label[train_idx]

# 테스트 데이터 설정
x_test = normalize_sentence[test_idx]
y_test = label[test_idx]

# 학습 데이터와 테스트 데이터의 크기 출력
print('len(x_train): ', len(x_train))
print('len(x_test): ', len(x_test))

In [ ]:
# 서브플롯 생성
fig, ax = plt.subplots(1, 2, figsize=(10, 5))

# 학습 데이터셋의 레이블 분포 시각화
x, y = np.unique(y_train, return_counts=True)
sns.barplot(x=x, y=y, ax=ax[0], palette=['red', 'green'])  # 부정적인 레이블은 빨간색, 긍정적인 레이블은 초록색
ax[0].set_title('Label Distribution in trainset')
ax[0].set_ylabel('Frequency')
ax[0].set_xticks(ticks=[0, 1], labels=['Negative', 'Positive'])
ax[0].bar_label(ax[0].containers[0], fmt='%d')
ax[0].bar_label(ax[0].containers[1], fmt='%d')
y_max = y.max()

# 테스트 데이터셋의 레이블 분포 시각화
x, y = np.unique(y_test, return_counts=True)
sns.barplot(x=x, y=y, ax=ax[1], palette=['red', 'green'])  # 부정적인 레이블은 빨간색, 긍정적인 레이블은 초록색
ax[1].set_title('Label Distribution in testset')
ax[1].set_ylabel('Frequency')
ax[1].set_xticks(ticks=[0, 1], labels=['Negative', 'Positive'])
ax[1].bar_label(ax[1].containers[0], fmt='%d')
ax[1].bar_label(ax[1].containers[1], fmt='%d')
if y_max < y.max():
    y_max = y.max()

# y축의 범위 설정
ax[0].set_ylim(0, y_max + 2000)
ax[1].set_ylim(0, y_max + 2000)

# 레이아웃 조정 및 그래프 출력
plt.tight_layout()
plt.show()

### Build Dataset

In [ ]:
class BERTDataset(Dataset):
    def __init__(self, inputs: np.ndarray, targets: np.ndarray, tokenizer: AutoTokenizer, max_seq_length: int = 100):
        """
        BERTDataset 초기화
        :param inputs: 입력 문장 배열
        :param targets: 출력 레이블 배열
        :param tokenizer: 사전 학습된 토크나이저
        :param max_seq_length: 최대 시퀀스 길이
        """
        self.max_seq_length = max_seq_length
        self.tokenizer = tokenizer
        self.inputs = inputs  # input sentence는 input 혹은 source라는 단어로 표현됨
        self.targets = targets  # output label은 target으로 표현됨

        # vocab 설정
        self.vocab = self.tokenizer.vocab
        self.pad_idx = self.vocab['[PAD]']
        self.cls_idx = self.vocab['[CLS]']
        self.sep_idx = self.vocab['[SEP]']
        self.mask_idx = self.vocab['[MASK]']

    def convert_inputs_to_feature(self, input):
        """
        입력 문장을 BERT 모델이 이해할 수 있는 형태로 변환
        :param input: 입력 문장
        :return: 토큰화된 입력 문장
        """
        # STEP 1: 문장을 토큰 단위로 분리한 후 숫자로 변환
        tokenized_input = self.tokenizer.encode(input)

        # STEP 2: 패딩 또는 토큰 자르기
        diff = self.max_seq_length - len(tokenized_input)
        if diff > 0:
            tokenized_input += [self.pad_idx] * diff
        else:
            tokenized_input = tokenized_input[:self.max_seq_length-1] + [self.sep_idx]

        return tokenized_input

    def idx2mask(self, token_ids):
        """
        어텐션 마스크 생성
        :param token_ids: 토큰 ID 리스트
        :return: 어텐션 마스크 리스트
        """
        return [token_id != self.pad_idx for token_id in token_ids]

    def __len__(self):
        """
        데이터셋의 길이 반환
        """
        return len(self.inputs)

    def __getitem__(self, idx):
        """
        주어진 인덱스에 해당하는 데이터 반환
        :param idx: 인덱스
        :return: 토큰화된 입력 문장, 어텐션 마스크, 레이블로 구성된 배치
        """
        input = self.inputs[idx]
        target = self.targets[idx]

        tokenized_input = self.convert_inputs_to_feature(input)
        att_mask = self.idx2mask(tokenized_input)

        # 토큰화된 입력과 어텐션 마스크의 길이가 max_seq_length와 같은지 체크
        assert len(tokenized_input) == self.max_seq_length
        assert len(att_mask) == self.max_seq_length

        batch = [
            torch.tensor(tokenized_input),
            torch.tensor(att_mask),
            torch.tensor(target),
        ]
        return batch

In [ ]:
# BERTDataset 인스턴스 생성
trainset = BERTDataset(inputs=x_train, targets=y_train, tokenizer=tokenizer, max_seq_length=cfg.max_seq_length)
testset = BERTDataset(inputs=x_test, targets=y_test, tokenizer=tokenizer, max_seq_length=cfg.max_seq_length)

# DataLoader 인스턴스 생성
trainloader = DataLoader(trainset, batch_size=cfg.batch_size, shuffle=True, num_workers=cfg.num_workers)
testloader = DataLoader(testset, batch_size=cfg.test_batch_size, shuffle=False, num_workers=cfg.num_workers)

In [ ]:
# 학습 데이터셋에서 패딩, CLS, SEP, MASK 토큰의 인덱스를 출력
print('[PAD] index: ', trainset.pad_idx)
print('[CLS] index: ', trainset.cls_idx)
print('[SEP] index: ', trainset.sep_idx)
print('[MASK] index: ', trainset.mask_idx)

In [ ]:
# 학습 데이터셋에서 첫 번째 문장 출력
print('[Sentence]')
print(trainset.inputs[0])

# 첫 번째 문장의 토큰화된 입력 출력
print('\n[Tokenized input]')
print('size: ', len(trainset[0][0]))
print(trainset[0][0])

# 첫 번째 문장의 어텐션 마스크 출력
print('\n[Attention mask]')
print('size: ', len(trainset[0][1]))
print(trainset[0][1])

# 첫 번째 문장의 타깃 레이블 출력
print('\n[Target]')
print(trainset[0][2])

## BERT for sequence classification

In [ ]:
# 사전 학습된 BERT 모델 로드
model = BertForSequenceClassification.from_pretrained(cfg.pretrained_name)

# 모델을 지정된 디바이스로 이동
model = model.to(cfg.device)

# 옵티마이저 설정
optimizer = AdamW(model.parameters(), lr=cfg.learning_rate)

In [ ]:
# 시드 설정
torch_seed(cfg.seed)

# 모델 학습 및 평가
train_history, test_history = fit(
    model        = model,        # 학습시킬 모델
    trainloader  = trainloader,  # 학습 데이터 로더
    testloader   = testloader,   # 테스트 데이터 로더
    optimizer    = optimizer,    # 옵티마이저
    epochs       = cfg.epochs,   # 에포크 수
    log_interval = cfg.log_interval, # 로그 출력 간격
    device       = cfg.device    # 사용할 디바이스
)

In [ ]:
def figure(all_train_history: list, all_test_history: list, all_exp_name: list) -> None:
    """
    성능 지표 시각화 함수
    :param all_train_history: 모든 실험의 학습 데이터 성능 지표 기록
    :param all_test_history: 모든 실험의 테스트 데이터 성능 지표 기록
    :param all_exp_name: 모든 실험의 이름 리스트
    """
    fig, ax = plt.subplots(2, 6, figsize=(20, 7))

    # 학습 데이터 성능 지표 라인 플롯
    for train_h, exp_name in zip(all_train_history, all_exp_name):
        for i, (k, v) in enumerate(train_h.items()):
            sns.lineplot(
                x=range(1, len(v) + 1),
                y=v,
                label=exp_name,
                ax=ax[0, i]
            )
            # y축 레이블 설정
            ax[0, i].set_ylabel(f'{k.upper()}')
            # x축 레이블 설정
            ax[0, i].set_xlabel('Epochs')
            # 제목 설정
            ax[0, i].set_title(f'{k.upper()} of Trainset')
            # 범례 설정
            ax[0, i].legend(loc='lower right')

    # 테스트 데이터 성능 지표 라인 플롯
    for test_h, exp_name in zip(all_test_history, all_exp_name):
        for i, (k, v) in enumerate(test_h.items()):
            sns.lineplot(
                x=range(1, len(v) + 1),
                y=v,
                label=exp_name,
                ax=ax[1, i]
            )
            # y축 레이블 설정
            ax[1, i].set_ylabel(f'{k.upper()}')
            # x축 레이블 설정
            ax[1, i].set_xlabel('Epochs')
            # 제목 설정
            ax[1, i].set_title(f'{k.upper()} of Testset')
            # 범례 설정
            ax[1, i].legend(loc='lower right')

    # 레이아웃 조정 및 그래프 출력
    plt.tight_layout()
    plt.show()

In [ ]:
# 학습 및 테스트 데이터 성능 지표 기록
all_train_history = [train_history]
all_test_history = [test_history]
all_exp_name = ['BERT']

# 성능 지표 시각화
figure(
    all_train_history=all_train_history,
    all_test_history=all_test_history,
    all_exp_name=all_exp_name
)

# Inference

---

In [ ]:
def predict(model, tokenizer, text, max_seq_length=100, device='cpu'):
    model.to(device)
    inputs = [text]
    targets = [0]  # 임의의 타겟 값, 실제로는 필요하지 않음

    dataset = BERTDataset(inputs=np.array(inputs), targets=np.array(targets), tokenizer=tokenizer, max_seq_length=max_seq_length)
    dataloader = DataLoader(dataset, batch_size=1)

    model.eval()
    with torch.no_grad():
        for batch in dataloader:
            input_ids, att_mask, _ = batch
            input_ids, att_mask = input_ids.to(device), att_mask.to(device)

            outputs = model(input_ids, attention_mask=att_mask)
            logits = outputs[0]
            probs = torch.softmax(logits, dim=1).cpu().numpy()

            pred = np.argmax(probs, axis=1).item()
            return pred, probs

In [ ]:
ind = 0
print(f"입력 텍스트: {testset.inputs[ind]}")
print(f"예측 레이블: {'긍정' if predict(model, tokenizer, testset.inputs[ind])[0]==0 else '부정'}")

In [ ]:
ind = 2
print(f"입력 텍스트: {testset.inputs[ind]}")
print(f"예측 레이블: {'긍정' if predict(model, tokenizer, testset.inputs[ind])[0]==0 else '부정'}")

---